# 0.0. __INIT__

## 0.1. Imports

In [1]:
import pandas as pd
import plotly.express as px

## 0.2. Helper Functions

In [2]:
PXChart = px._chart_types

def numerical_description(numerical: pd.DataFrame) -> pd.DataFrame:
    description = numerical.describe().T
    description['range'] = description['max'] - description['min']
    description['skew'] = numerical.skew()
    description['kurtosis'] = numerical.kurtosis()

    return description

def plot_histogram(data: pd.DataFrame, column: str) -> None:
    fig = px.histogram(data, x=column)
    fig = fig.update_layout(
        title_text=f'{snake_to_text(column)} histogram',
        xaxis_title_text='Value',
        yaxis_title_text='Count')
    fig.show()

def plot_xy(data: pd.DataFrame, x_axis: str, y_axis: str, plot_type: PXChart) -> None:
    fig = plot_type(data, x=x_axis, y=y_axis)
    fig = fig.update_layout(
        title_text=f'{snake_to_text(y_axis)}',
        xaxis_title_text=snake_to_text(x_axis),
        yaxis_title_text='Value')
    fig.show()

def snake_to_text(string: str) -> str:
    return string.capitalize().replace("_", " ")

## 0.3. Loading data

In [3]:
data_metrics = pd.read_excel("data/Data_2021.xls")
data_historic = pd.read_excel("data/Historic_data.xls")

# 1.0. DATA DESCRIPTION

In [85]:
df1_metrics = data_metrics.copy()
df1_historic = data_historic.copy()

## 1.1. Metrics description

In [21]:
df1_metrics.head()

,Country name,Regional indicator,Ladder score,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,Finland,Western Europe,7.8421,10.775202,0.953603,72.000000,0.949268,-0.097760,0.185846
1,Denmark,Western Europe,7.6195,10.933176,0.954410,72.699753,0.945639,0.030109,0.178838
2,Switzerland,Western Europe,7.5715,11.117368,0.941742,74.400101,0.918788,0.024629,0.291698
3,Iceland,Western Europe,7.5539,10.877768,0.982938,73.000000,0.955123,0.160274,0.672865
4,Netherlands,Western Europe,7.4640,10.931812,0.941601,72.400116,0.913116,0.175404,0.337938


### 1.1.0. Renaming columns

In [22]:
df1_metrics.columns = [name.lower().replace(" ", "_") for name in df1_metrics.columns]

### 1.1.1. Data Dimensions

In [23]:
print(f'Number of rows: {df1_metrics.shape[0]}')
print(f'Number of columns: {df1_metrics.shape[1]}')

Number of rows: 149
Number of columns: 9


### 1.1.2. Data types

In [24]:
df1_metrics.dtypes

country_name                     object
regional_indicator               object
ladder_score                    float64
logged_gdp_per_capita           float64
social_support                  float64
healthy_life_expectancy         float64
freedom_to_make_life_choices    float64
generosity                      float64
perceptions_of_corruption       float64
dtype: object

### 1.1.3. Check NA

In [25]:
df1_metrics.isna().sum()

country_name                    0
regional_indicator              0
ladder_score                    0
logged_gdp_per_capita           0
social_support                  0
healthy_life_expectancy         0
freedom_to_make_life_choices    0
generosity                      0
perceptions_of_corruption       0
dtype: int64

### 1.1.4. Fillout NA

In [26]:
# There is no NA
pass

### 1.1.5 Change types

In [27]:
# Not needed
pass

### 1.1.6. Descriptive Statistical

In [28]:
numerical = df1_metrics.select_dtypes(include=['int64', 'float64'])
categorical = df1_metrics.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]'])

#### 1.1.6.1. Numerical attributes

In [29]:
numerical_description(numerical)

,count,mean,std,min,25%,50%,75%,max,range,skew,kurtosis
ladder_score,149.0,5.532832,1.073923,2.522900,4.851800,5.533500,6.255400,7.842100,5.319200,-0.104272,-0.368236
logged_gdp_per_capita,149.0,9.432209,1.158585,6.635322,8.540956,9.568728,10.421198,11.646564,5.011242,-0.352070,-0.815365
social_support,149.0,0.814726,0.114892,0.462596,0.749777,0.831914,0.904702,0.982938,0.520342,-0.937962,0.398104
healthy_life_expectancy,149.0,64.992784,6.762071,48.478001,59.801552,66.603096,69.599998,76.952858,28.474857,-0.522006,-0.564196
freedom_to_make_life_choices,149.0,0.791572,0.113315,0.381749,0.718051,0.804414,0.877263,0.970131,0.588382,-0.756627,0.412602
generosity,149.0,-0.015149,0.150653,-0.288153,-0.126469,-0.036025,0.078830,0.541553,0.829706,1.009203,1.631913
perceptions_of_corruption,149.0,0.727477,0.179266,0.081959,0.666607,0.780623,0.844619,0.939343,0.857385,-1.576883,2.248839


In [30]:
for column in numerical.columns:
    plot_histogram(numerical, column)

#### 1.1.6.2. Categorical attributes

In [31]:
# Checking how many categories in each column
categorical.apply(lambda x: x.unique().shape[0])

country_name          149
regional_indicator     10
dtype: int64

In [32]:
# Bar plot for smallest granularity
for numerical_variable in numerical.columns:
    df_auxiliar = df1_metrics.sort_values(numerical_variable)
    plot_xy(df_auxiliar, x_axis="country_name", y_axis=numerical_variable, plot_type=px.bar)

In [33]:
for numerical_variable in numerical.columns:
    df_auxiliar = df1_metrics.sort_values(numerical_variable)
    plot_xy(df_auxiliar, x_axis="regional_indicator", y_axis=numerical_variable, plot_type=px.box)

## 1.2. Historic description

In [86]:
df1_historic.head()

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
0,Afghanistan,2008,3.723590,7.370100,0.450662,50.799999,0.718114,0.167640,0.881686,0.517637,0.258195
1,Afghanistan,2009,4.401778,7.539972,0.552308,51.200001,0.678896,0.190099,0.850035,0.583926,0.237092
2,Afghanistan,2010,4.758381,7.646709,0.539075,51.599998,0.600127,0.120590,0.706766,0.618265,0.275324
3,Afghanistan,2011,3.831719,7.619532,0.521104,51.919998,0.495901,0.162427,0.731109,0.611387,0.267175
4,Afghanistan,2012,3.782938,7.705479,0.520637,52.240002,0.530935,0.236032,0.775620,0.710385,0.267919


### 1.2.0. Rename columns

In [87]:
df1_historic.columns = [name.lower().replace(" ", "_") for name in df1_historic.columns]

### 1.2.1. Data Dimensions

In [88]:
print(f'Number of rows: {df1_historic.shape[0]}')
print(f'Number of columns: {df1_historic.shape[1]}')

Number of rows: 1949
Number of columns: 11


### 1.2.2. Data types

In [89]:
df1_historic.dtypes

country_name                         object
year                                  int64
life_ladder                         float64
log_gdp_per_capita                  float64
social_support                      float64
healthy_life_expectancy_at_birth    float64
freedom_to_make_life_choices        float64
generosity                          float64
perceptions_of_corruption           float64
positive_affect                     float64
negative_affect                     float64
dtype: object

### 1.2.3. Check NA

In [90]:
df1_historic.isna().sum()

country_name                          0
year                                  0
life_ladder                           0
log_gdp_per_capita                   36
social_support                       13
healthy_life_expectancy_at_birth     55
freedom_to_make_life_choices         32
generosity                           89
perceptions_of_corruption           110
positive_affect                      22
negative_affect                      16
dtype: int64

In [91]:
number_of_countries = df1_historic["country_name"].unique().shape[0]
df_auxiliar = df1_historic.groupby("year").count() / number_of_countries * 100
df_auxiliar.sort_values("positive_affect").reset_index()

,year,country_name,life_ladder,log_gdp_per_capita,social_support,healthy_life_expectancy_at_birth,freedom_to_make_life_choices,generosity,perceptions_of_corruption,positive_affect,negative_affect
0,2005,16.265060,16.265060,16.265060,16.265060,16.265060,15.060241,0.602410,15.060241,15.662651,15.662651
1,2006,53.614458,53.614458,53.012048,53.614458,53.012048,51.807229,48.795181,50.000000,53.614458,53.012048
2,2020,57.228916,57.228916,53.012048,57.228916,55.421687,56.626506,53.012048,53.614458,56.626506,56.626506
3,2007,61.445783,61.445783,61.445783,60.240964,60.843373,60.843373,61.445783,59.036145,60.843373,61.445783
4,2008,66.265060,66.265060,66.265060,66.265060,65.060241,64.457831,65.662651,64.457831,65.060241,66.265060
5,2009,68.674699,68.674699,68.072289,68.072289,66.867470,67.469880,68.072289,66.867470,68.072289,68.072289
6,2010,74.698795,74.698795,74.096386,72.289157,72.891566,73.493976,74.096386,71.686747,72.289157,72.289157
7,2013,82.530120,82.530120,81.927711,82.530120,80.120482,81.927711,80.120482,78.313253,81.325301,82.530120
8,2018,85.542169,85.542169,81.927711,85.542169,83.132530,84.939759,81.927711,80.722892,84.337349,84.337349
9,2014,87.349398,87.349398,85.542169,85.542169,84.337349,83.132530,83.132530,81.927711,84.939759,85.542169


In [92]:
number_of_years = df1_historic["year"].unique().shape[0]
df_auxiliar = df1_historic.groupby("country_name").count() / number_of_years * 100
df_auxiliar.sort_values("positive_affect").reset_index()

,country_name,year,life_ladder,log_gdp_per_capita,social_support,healthy_life_expectancy_at_birth,freedom_to_make_life_choices,generosity,perceptions_of_corruption,positive_affect,negative_affect
0,Maldives,6.25,6.25,6.25,6.25,6.25,6.25,6.25,0.00,0.00,0.00
1,Oman,6.25,6.25,6.25,0.00,6.25,6.25,6.25,0.00,0.00,6.25
2,Cuba,6.25,6.25,0.00,6.25,6.25,6.25,0.00,0.00,6.25,6.25
3,Guyana,6.25,6.25,6.25,6.25,6.25,6.25,6.25,6.25,6.25,6.25
4,Suriname,6.25,6.25,6.25,6.25,6.25,6.25,6.25,6.25,6.25,6.25
...,...,...,...,...,...,...,...,...,...,...,...
161,South Korea,93.75,93.75,93.75,93.75,93.75,93.75,87.50,93.75,93.75,93.75
162,Spain,93.75,93.75,93.75,93.75,93.75,93.75,87.50,93.75,93.75,93.75
163,Georgia,93.75,93.75,93.75,93.75,93.75,93.75,93.75,93.75,93.75,93.75
164,Russia,93.75,93.75,93.75,93.75,93.75,93.75,93.75,93.75,93.75,93.75


### 1.2.4. Fillout NA

In [98]:
# For simplicity -- Considering that in section 1.2.6.2. outliers wasn't observed in a large amount (by looking only)
not_filled_df = df1_historic.copy()
for column in df1_historic.columns[1:]:
    df1_historic[column].fillna(df1_historic[column].median(), inplace=True)

In [99]:
df1_historic.isna().sum()

country_name                        0
year                                0
life_ladder                         0
log_gdp_per_capita                  0
social_support                      0
healthy_life_expectancy_at_birth    0
freedom_to_make_life_choices        0
generosity                          0
perceptions_of_corruption           0
positive_affect                     0
negative_affect                     0
dtype: int64

### 1.2.5 Change types

In [100]:
# Not needed
pass

### 1.2.6. Descriptive Statistical

In [101]:
numerical = df1_historic.select_dtypes(include=['int64', 'float64'])
categorical = df1_historic.select_dtypes(exclude=['float64', 'datetime64[ns]'])  # Excluded int64

#### 1.2.6.1. Numerical attributes

In [102]:
numerical_description(numerical)

,count,mean,std,min,25%,50%,75%,max,range,skew,kurtosis
year,1949.0,2013.216008,4.166828,2005.000000,2010.000000,2013.000000,2017.000000,2020.000000,15.000000,-0.128238,-1.077674
life_ladder,1949.0,5.466707,1.115717,2.375092,4.640079,5.386025,6.283498,8.018934,5.643842,0.068491,-0.688600
log_gdp_per_capita,1949.0,9.370155,1.143444,6.635322,8.478379,9.460323,10.335316,11.648169,5.012846,-0.315749,-0.821982
social_support,1949.0,0.812704,0.118099,0.290184,0.750738,0.835167,0.904971,0.987343,0.697159,-1.117841,1.216308
healthy_life_expectancy_at_birth,1949.0,63.411317,7.409737,32.299999,58.900002,65.199997,68.400002,77.099998,44.799999,-0.773842,0.043662
freedom_to_make_life_choices,1949.0,0.742910,0.140957,0.257534,0.649316,0.763476,0.853534,0.985178,0.727644,-0.635143,-0.074113
generosity,1949.0,-0.001056,0.158561,-0.335040,-0.106459,-0.025393,0.084833,0.698099,1.033139,0.846856,1.040651
perceptions_of_corruption,1949.0,0.750233,0.181891,0.035198,0.698808,0.802428,0.867640,0.983276,0.948078,-1.578747,2.185235
positive_affect,1949.0,0.710138,0.106508,0.321690,0.627351,0.722391,0.797785,0.943621,0.621931,-0.369953,-0.555219
negative_affect,1949.0,0.268466,0.084830,0.082737,0.206641,0.258117,0.319231,0.704590,0.621853,0.743382,0.899204


In [103]:
for column in numerical.columns:
    plot_histogram(numerical, column)

#### 1.2.6.2. Categorical attributes

In [104]:
# Checking how many categories in each column
categorical.apply(lambda x: x.unique().shape[0])

country_name    166
year             16
dtype: int64

In [105]:
# Bar plot for smallest granularity
for numerical_variable in numerical.columns[1:]:  # To exclude "year" column
    df_auxiliar = df1_historic.sort_values(numerical_variable)
    plot_xy(df_auxiliar, x_axis="country_name", y_axis=numerical_variable, plot_type=px.box)

# 2.0. FEATURE ENGENEERING

## 2.1. Hypothesis creation

### 2.1.1. Store Hypothesis

## 2.3. Featuring engineering

# 3.0. DATA FILTERING

## 3.1. Lines filtering

## 3.2. Columns filtering

# 4.0. EXPLORATORY DATA ANALISYS (EDA)

## 4.1. Univariate Analysis

### 4.1.1. Response Variable

### 4.1.2. Numerical Variables

### 4.1.3. Categorical Variables

## 4.2. Bivariate Analysis

### **H1.** Lojas com maior sortimento deveriam vender mais.
**FALSA** Lojas com MAIOR SORTIMENTO vendem MENOS

### **H2.** Lojas com competidores mais perto deveriam vender mais.
**VERDADEIRA**

### **H3.** Lojas com competidores há mais tempo deveriam vender mais.
**FALSA** Lojas com competidores há MAIS TEMPO vendem MENOS

### **H4.** Lojas com promoções ativas por mais tempo deveriam vender mais.
**FALSA** Lojas com promoções ativas por MAIS TEMPO vendem MENOS depois de um certo período de tempo

### 4.2.1. Hypothesis summary

## 4.3. Multivariate Analysis

### 4.3.1 Numerical Attributes

### 4.3.2 Categorical Attributes

# 5.0. DATA PREPARATION

## 5.1. Normalization
Because it was not possible to identify any normal distribution in section 4.1.2., there will be no data normalization.

## 5.2. Rescaling

## 5.3. Transformation

### 5.3.1. Encoding

### 5.3.2. Response variable transformation

### 5.3.3. Nature transformation

# 6.0. FEATURE SELECTION

## 6.1. Split dataframe into training and test datasets

## 6.2. Feature Selector

### 6.2.1. Best Features

## 6.3. Feature Selection

# 7.0. MACHINE LEARNING MODELLING

## 7.1. Baseline Model

## 7.2. Linear Regression Model

### 7.2.1. Linear Regression Model - Cross Validation

## 7.3. Linear Regression Regularized Model - Lasso

### 7.3.1. Linear Regression Regularized Model - Cross Validation

## 7.4. Random Forest Regressor

### 7.4.1. Random Forest Regressor - Cross Validation

## 7.5. XGBoost Regressor

### 7.5.1. XGBoost Regressor - Cross Validation

## 7.6. Model comparison

### 7.6.1. Single Performance

### 7.6.2. Real Performance - Cross Validation

# 8.0. HYPERPARAMETER FINE TUNING


## 8.1. Random Search

## 8.2. Final Model

# 9.0. ERROR TRANSLATION AND INTERPRETATION


## 9.1. Business Performance

## 9.2. Total Performance

## 9.3. Machine Learning Performance

# 10.0. DEPLOY MODEL TO PRODUCTION


## 10.1. Rossmann Class

## 10.2. API Handler

## 10.3. API Tester